# DBScan

**Exercise:** [![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/kks32/pcdemo/blob/main/lite/content/dbscan.ipynb)

### Get data

In [7]:
!mkdir -p data/
!wget -P data/ https://github.com/prof-rausch/pcdemo/raw/refs/heads/main/lite/content/data/points_small.npz data/

--2025-11-10 14:48:42--  https://github.com/prof-rausch/pcdemo/raw/refs/heads/main/lite/content/data/points_small.npz
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/prof-rausch/pcdemo/refs/heads/main/lite/content/data/points_small.npz [following]
--2025-11-10 14:48:42--  https://raw.githubusercontent.com/prof-rausch/pcdemo/refs/heads/main/lite/content/data/points_small.npz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13466 (13K) [application/octet-stream]
Saving to: ‘data/points_small.npz’

points_small.npz    100%[===================>]  13.15K  --.-KB/s    in 0s      

2025-11-10 14:48:42 (

In [8]:
import numpy as np, plotly.graph_objects as go

data = np.load("data/points_small.npz")
pts = data["points"]  # shape (N,3)

z = pts[:,2]
# normalize 0..1 for coloring
zmin, zmax = float(z.min()), float(z.max())
zn = (z - zmin) / (zmax - zmin + 1e-9)

fig = go.Figure(data=[go.Scatter3d(
    x=pts[:,0], y=pts[:,1], z=pts[:,2],
    mode="markers",
    marker=dict(size=2, color=zn)  # default colorscale
)])
fig.update_layout(height=500, margin=dict(l=0,r=0,b=0,t=30), title="Colored by z")
fig.show()

In [9]:
import numpy as np
import plotly.graph_objects as go

pts = np.load("data/points_small.npz")["points"]

# --- Minimal DBSCAN‑ish (radius graph + core expansion); for teaching only ---
def dbscan_numpy(X, eps=0.15, min_pts=12):
    N = len(X); labels = -np.ones(N, dtype=int)
    visited = np.zeros(N, dtype=bool)
    cluster_id = 0

    # precompute simple radius neighbors (O(N^2) for tiny sets)
    d2 = ((X[:,None,:]-X[None,:,:])**2).sum(axis=2)
    rad = d2 <= (eps**2)

    for i in range(N):
        if visited[i]:
            continue
        visited[i] = True
        neighbors = np.where(rad[i])[0]
        if neighbors.size < min_pts:
            labels[i] = -1  # noise
        else:
            # start new cluster
            labels[i] = cluster_id
            seeds = set(neighbors.tolist())
            seeds.discard(i)
            while seeds:
                j = seeds.pop()
                if not visited[j]:
                    visited[j] = True
                    nbs = np.where(rad[j])[0]
                    if nbs.size >= min_pts:
                        seeds.update(set(nbs.tolist()))
                if labels[j] == -1:
                    labels[j] = cluster_id
                if labels[j] == -1 or labels[j] == -1:  # keep assigned
                    labels[j] = cluster_id
            cluster_id += 1
    return labels

labels = dbscan_numpy(pts, eps=0.18, min_pts=15)
n_clusters = labels.max() + 1
n_noise = (labels == -1).sum()
print(f"clusters: {n_clusters}, noise: {n_noise}")

# visualize
# map labels -> scalar colors (noise -> -1)
colors = labels.astype(float)
fig = go.Figure(data=[go.Scatter3d(
    x=pts[:,0], y=pts[:,1], z=pts[:,2],
    mode="markers",
    marker=dict(size=2, color=colors)
)])
fig.update_layout(height=500, margin=dict(l=0,r=0,b=0,t=30), title="DBSCAN‑like clustering (NumPy)")
fig.show()

clusters: 2, noise: 501
